In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!mkdir utils

In [3]:
#import torch

from utils.dataset_utils import Human36
from model_architecture import MotionMixerCNN
from utils.utils import model_params, smooth, save_model, load_model, set_random_seed

### Initialize Dataset

In [4]:
path = '/content/drive/MyDrive/FinalProject/h3.6m'

valid_dataset = Human36(path, d_mode = 'valid', seed_length = 10, prediction_length = 10)
train_dataset = valid_dataset #Human36(path, d_mode = 'train', seed_length = 10, prediction_length = 10)
test_dataset = valid_dataset #Human36(path, d_mode = 'test', seed_length = 10, prediction_length = 10)

In [5]:
import torch.utils.data as data

valid_dataset = data.Subset(valid_dataset, range(100))
train_dataset = data.Subset(train_dataset, range(100))
test_dataset = data.Subset(test_dataset, range(100))

print(test_dataset.__len__())
print(valid_dataset.__len__())
print(train_dataset.__len__())

100
100
100


In [6]:
item = valid_dataset.__getitem__(0)

print(item[0].shape)
print(item[1].shape)

torch.Size([10, 99])
torch.Size([10, 99])


In [7]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

### Init Model

In [8]:
motionmixer = MotionMixerCNN(input_size_spatial = 99,
               input_size_temporal = 10,
               embedding_size = 64,
               n_block = 3,
               n_output = 10,
               n_temp_layers = 2,
               n_spatial_layers = 2,
               temp_kernel_size = 5,
               spatial_kernel_size = 15)

print(f"The model has {model_params(motionmixer)} parameter.")

The model has 13965 parameter.


# Trainer

In [9]:
import os
import shutil
from torch.utils.tensorboard import SummaryWriter

TBOARD_LOGS = os.path.join(os.getcwd(), "tboard_logs", "motionMixer")
if not os.path.exists(TBOARD_LOGS):
    os.makedirs(TBOARD_LOGS)

shutil.rmtree(TBOARD_LOGS)
writer = SummaryWriter(TBOARD_LOGS)

In [10]:
SAVE_PATH = '/content/test'

from utils.train import Trainer

trainer = Trainer(model = motionmixer,
                    NUM_EPOCHS  = 100,
                    train_loader = train_loader,
                    valid_loader = test_loader,
                    EVAL_FREQ = 2,
                    SAVE_FREQ = 1,
                    savepath = SAVE_PATH,
                    writer = writer,
                    lr = 2e-3,
                    gamma = 0.1,
                    step_size = 5,
                    loadpath = None)

In [11]:
stats, model =  trainer.train()

Ep 100: Iter 4/4: Loss=nan: 100%|██████████| 100/100 [00:22<00:00,  4.36it/s]
